In [1]:
from __future__ import print_function

import os
import sys
import numpy as np

from keras.optimizers import SGD, Nadam, RMSprop
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.regularizers import l1, l2

sys.path.append(os.path.join(os.getcwd(), os.pardir))

import config

from utils.dataset.data_generator import DataGenerator
from utils.training.callbacks import Logger
from models.cnn3 import cnn, cnn_regularized

Using Theano backend.
Using gpu device 0: GeForce GTX 680 (CNMeM is disabled, cuDNN 5005)


In [3]:
lr=0.01
decay = 0.0015
l1 = 0.00001
l2 = 0.00001
dropout = 0.5
n_epochs=500
batch_size=32
input_shape=(140, 140, 3)
weights='cnn_140_rgb_corrected_lr_0.005000_sgd_he_normal__l1_0.000010_l2_0.000010_dropout_0.500000_r_training_weights_best.hdf5'

name = 'cnn_140_rgb_corrected_full_lr_%f_decay_%f_sgd_he_normal__l1_%f_l2_%f_dropout_%f_r' % (lr, decay, l1, l2, dropout)

In [4]:
print('loading model...')
# model = cnn(input_shape=input_shape, init='he_normal')
model = cnn_regularized(input_shape=input_shape, init='he_normal', l1=l1, l2=l2)
model.summary()

optimizer = SGD(lr=lr, clipnorm=4., nesterov=True, decay=decay)
# optimizer = Nadam(lr=lr)
# optimizer = RMSprop(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

logger = Logger('%s_training_batch.log' % name, append=False)
csv_logger = CSVLogger('%s_training.log' % name, append=False)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

current_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_current.hdf5" % name), verbose=0)

loading model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 140, 140, 128) 18944       input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 128) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 70, 70, 128)   0           activation_1[0][0]               
__________________________________________________________________________

In [5]:
print('Initializing data generators...')
train_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/train_data.csv'
validation_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/validation_data.csv'
test_set_file = '/home/tanuj/Workspace/power-grid-detection/dataset/corrected/19/test_data.csv'

train_data_gen = DataGenerator(dataset_file=train_set_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=validation_set_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=test_set_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [6]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, logger, best_model_checkpointer, current_model_checkpointer],
                              nb_worker=4, ep)
print('done.')

Fitting model...
Epoch 1/500
65504/65504 [==============================] - 2227s - loss: 2.4611 - acc: 0.9094 - val_loss: 0.0885 - val_acc: 0.9689
Epoch 2/500
65504/65504 [==============================] - 2234s - loss: 2.2601 - acc: 0.9729 - val_loss: 0.0679 - val_acc: 0.9783
Epoch 3/500
65504/65504 [==============================] - 2236s - loss: 2.2424 - acc: 0.9771 - val_loss: 0.0653 - val_acc: 0.9789
Epoch 4/500
65504/65504 [==============================] - 2236s - loss: 2.2357 - acc: 0.9781 - val_loss: 0.0652 - val_acc: 0.9790
Epoch 5/500
65504/65504 [==============================] - 2238s - loss: 2.2311 - acc: 0.9783 - val_loss: 0.0630 - val_acc: 0.9797
Epoch 6/500
65504/65504 [==============================] - 2237s - loss: 2.2259 - acc: 0.9790 - val_loss: 0.0624 - val_acc: 0.9802
Epoch 7/500
65504/65504 [==============================] - 2238s - loss: 2.2226 - acc: 0.9796 - val_loss: 0.0616 - val_acc: 0.9803
Epoch 8/500
65504/65504 [==============================] - 2235s -

KeyboardInterrupt: 

In [7]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])

Evaluating model...
done.
Test score: 0.055229082056
Test accuracy: 0.98305808656
